In [1]:
import gc
import os
from datetime import datetime
import pickle
import psutil
from tqdm import tqdm, tqdm_notebook
from functools import partial, update_wrapper
from itertools import product
from multiprocessing import Pool

import bloscpack as bp

import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

from YSMLT import utils as g_utils
from YSMLT.series import utils as ts_utils

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy.signal import welch, find_peaks
from scipy import stats
from scipy.special import entr
from scipy.stats import entropy
from tsfresh.feature_extraction import feature_calculators

### mean -0.31079195780498403
### std  2.4983141023158186

In [2]:
pdf_trn = pd.read_csv('../input/train_clean.csv')
pdf_tst = pd.read_csv('../input/test_clean.csv')

with open('../input/batch_ids_trn.pkl', 'rb') as f:
    batch_id_trn = pickle.load(f)
with open('../input/batch_ids_tst.pkl', 'rb') as f:
    batch_id_tst = pickle.load(f)

In [3]:
step = 500
wndw = 500
# fill = -0.31079195780498403

In [4]:
def wrapped_spkt_welch_density(srs, c): return list(feature_calculators.spkt_welch_density(srs, [{'coeff': c}]))[0][1]

In [5]:
for i in tqdm(range(0, 10)):
    sgnl_ndcs = batch_id_trn[i]
    
    trgt = pdf_trn['open_channels'].iloc[sgnl_ndcs]
    
    # ------------------------------------------------
    sgnl_p1 = pdf_trn['signal'].iloc[sgnl_ndcs]
    p1_fill = sgnl_p1.mean()
    
    sgnl_p1b5 = sgnl_p1.shift(-5, fill_value=p1_fill)
    sgnl_p1b4 = sgnl_p1.shift(-4, fill_value=p1_fill)
    sgnl_p1b3 = sgnl_p1.shift(-3, fill_value=p1_fill)
    sgnl_p1b2 = sgnl_p1.shift(-2, fill_value=p1_fill)
    sgnl_p1b1 = sgnl_p1.shift(-1, fill_value=p1_fill)
    sgnl_p1f1 = sgnl_p1.shift(1, fill_value=p1_fill)
    sgnl_p1f2 = sgnl_p1.shift(2, fill_value=p1_fill)
    sgnl_p1f3 = sgnl_p1.shift(3, fill_value=p1_fill)
    sgnl_p1f4 = sgnl_p1.shift(4, fill_value=p1_fill)
    sgnl_p1f5 = sgnl_p1.shift(5, fill_value=p1_fill)
    
    sgnl_stck0 = np.stack([sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1, sgnl_p1, sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5], -1)
    sgnl_stckf = np.stack([sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5], -1)
    sgnl_stckb = np.stack([sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1], -1)
    
    sgnl_allwelch_n4 = pd.Series(np.array([wrapped_spkt_welch_density(sub, -4) for sub in sgnl_stck0]))
    sgnl_allwelch_0 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 0) for sub in sgnl_stck0]))
    sgnl_allwelch_p4 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 4) for sub in sgnl_stck0]))
    sgnl_fwelch_n3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, -3) for sub in sgnl_stckf]))
    sgnl_fwelch_0 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 0) for sub in sgnl_stckf]))
    sgnl_fwelch_p3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 3) for sub in sgnl_stckf]))
    sgnl_bwelch_n3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, -3) for sub in sgnl_stckb]))
    sgnl_bwelch_0 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 0) for sub in sgnl_stckb]))
    sgnl_bwelch_p3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 3) for sub in sgnl_stckb]))
    
    sgnl_binent_0 = pd.Series(np.array([feature_calculators.binned_entropy(sub, max_bins=10) for sub in sgnl_stck0]))
    sgnl_binent_f = pd.Series(np.array([feature_calculators.binned_entropy(sub, max_bins=10) for sub in sgnl_stckf]))
    sgnl_binent_b = pd.Series(np.array([feature_calculators.binned_entropy(sub, max_bins=10) for sub in sgnl_stckb]))

    sr_p1b5 = ts_utils.serie_stacker(sgnl_p1b5, step_size=step, block_size=wndw)
    sr_p1b4 = ts_utils.serie_stacker(sgnl_p1b4, step_size=step, block_size=wndw)
    sr_p1b3 = ts_utils.serie_stacker(sgnl_p1b3, step_size=step, block_size=wndw)
    sr_p1b2 = ts_utils.serie_stacker(sgnl_p1b2, step_size=step, block_size=wndw)
    sr_p1b1 = ts_utils.serie_stacker(sgnl_p1b1, step_size=step, block_size=wndw)
    sr_p1  = ts_utils.serie_stacker(sgnl_p1, step_size=step, block_size=wndw)
    sr_p1f1 = ts_utils.serie_stacker(sgnl_p1f1, step_size=step, block_size=wndw)
    sr_p1f2 = ts_utils.serie_stacker(sgnl_p1f2, step_size=step, block_size=wndw)
    sr_p1f3 = ts_utils.serie_stacker(sgnl_p1f3, step_size=step, block_size=wndw)
    sr_p1f4 = ts_utils.serie_stacker(sgnl_p1f4, step_size=step, block_size=wndw)
    sr_p1f5 = ts_utils.serie_stacker(sgnl_p1f5, step_size=step, block_size=wndw)
    
    sr_allwelch_n4 = ts_utils.serie_stacker(sgnl_allwelch_n4, step_size=step, block_size=wndw)
    sr_allwelch_0 = ts_utils.serie_stacker(sgnl_allwelch_0, step_size=step, block_size=wndw)
    sr_allwelch_p4 = ts_utils.serie_stacker(sgnl_allwelch_p4, step_size=step, block_size=wndw)
    sr_fwelch_n3 = ts_utils.serie_stacker(sgnl_fwelch_n3, step_size=step, block_size=wndw)
    sr_fwelch_0 = ts_utils.serie_stacker(sgnl_fwelch_0, step_size=step, block_size=wndw)
    sr_fwelch_p3 = ts_utils.serie_stacker(sgnl_fwelch_p3, step_size=step, block_size=wndw)
    sr_bwelch_n3 = ts_utils.serie_stacker(sgnl_bwelch_n3, step_size=step, block_size=wndw)
    sr_bwelch_0 = ts_utils.serie_stacker(sgnl_bwelch_0, step_size=step, block_size=wndw)
    sr_bwelch_p3 = ts_utils.serie_stacker(sgnl_bwelch_p3, step_size=step, block_size=wndw)
    
    sr_binent_0 = ts_utils.serie_stacker(sgnl_binent_0, step_size=step, block_size=wndw)
    sr_binent_f = ts_utils.serie_stacker(sgnl_binent_f, step_size=step, block_size=wndw)
    sr_binent_b = ts_utils.serie_stacker(sgnl_binent_b, step_size=step, block_size=wndw)
    
    # ------------------------------------------------
    sgnl_p2 = sgnl_p1**2
    p2_fill = sgnl_p2.mean()
    
    sgnl_p2b5 = sgnl_p2.shift(-5, fill_value=p2_fill)
    sgnl_p2b4 = sgnl_p2.shift(-4, fill_value=p2_fill)
    sgnl_p2b3 = sgnl_p2.shift(-3, fill_value=p2_fill)
    sgnl_p2b2 = sgnl_p2.shift(-2, fill_value=p2_fill)
    sgnl_p2b1 = sgnl_p2.shift(-1, fill_value=p2_fill)
    sgnl_p2f1 = sgnl_p2.shift(1, fill_value=p2_fill)
    sgnl_p2f2 = sgnl_p2.shift(2, fill_value=p2_fill)
    sgnl_p2f3 = sgnl_p2.shift(3, fill_value=p2_fill)
    sgnl_p2f4 = sgnl_p2.shift(4, fill_value=p2_fill)
    sgnl_p2f5 = sgnl_p2.shift(5, fill_value=p2_fill)
    
    sr_p2b5 = ts_utils.serie_stacker(sgnl_p2b5, step_size=step, block_size=wndw)
    sr_p2b4 = ts_utils.serie_stacker(sgnl_p2b4, step_size=step, block_size=wndw)
    sr_p2b3 = ts_utils.serie_stacker(sgnl_p2b3, step_size=step, block_size=wndw)
    sr_p2b2 = ts_utils.serie_stacker(sgnl_p2b2, step_size=step, block_size=wndw)
    sr_p2b1 = ts_utils.serie_stacker(sgnl_p2b1, step_size=step, block_size=wndw)
    sr_p2  = ts_utils.serie_stacker(sgnl_p2, step_size=step, block_size=wndw)
    sr_p2f1 = ts_utils.serie_stacker(sgnl_p2f1, step_size=step, block_size=wndw)
    sr_p2f2 = ts_utils.serie_stacker(sgnl_p2f2, step_size=step, block_size=wndw)
    sr_p2f3 = ts_utils.serie_stacker(sgnl_p2f3, step_size=step, block_size=wndw)
    sr_p2f4 = ts_utils.serie_stacker(sgnl_p2f4, step_size=step, block_size=wndw)
    sr_p2f5 = ts_utils.serie_stacker(sgnl_p2f5, step_size=step, block_size=wndw)
    
    sr = np.stack(
        [
            sr_p1, 
            sr_p1b5, sr_p1b4, sr_p1b3, sr_p1b2, sr_p1b1, sr_p1f1, sr_p1f2, sr_p1f3, sr_p1f4, sr_p1f5,
            sr_allwelch_n4, sr_allwelch_0, sr_allwelch_p4, sr_fwelch_n3, sr_fwelch_0, sr_fwelch_p3, sr_bwelch_n3, sr_bwelch_0, sr_bwelch_p3,
            sr_binent_0, sr_binent_f, sr_binent_b,
            sr_p2, 
            sr_p2b5, sr_p2b4, sr_p2b3, sr_p2b2, sr_p2b1, sr_p2f1, sr_p2f2, sr_p2f3, sr_p2f4, sr_p2f5,
        ],
        axis=2
    )
    
    tr = ts_utils.serie_stacker(trgt, step_size=step, block_size=wndw)
    
    bp.pack_ndarray_to_file(sr, '../input/trn_srs_dat_g{:d}_s{:d}_w{:d}_feat_basicwithnew.bp'.format(i, step, wndw))
    bp.pack_ndarray_to_file(tr, '../input/trn_srs_lbl_g{:d}_s{:d}_w{:d}_feat_basicwithnew.bp'.format(i, step, wndw))
    
    del sgnl_ndcs
    
    del sgnl_p1, sgnl_p2
    del sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1, sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5
    del sgnl_p2b5, sgnl_p2b4, sgnl_p2b3, sgnl_p2b2, sgnl_p2b1, sgnl_p2f1, sgnl_p2f2, sgnl_p2f3, sgnl_p2f4, sgnl_p2f5
    
    del sgnl_allwelch_n4, sgnl_allwelch_0, sgnl_allwelch_p4, sgnl_fwelch_n3, sgnl_fwelch_0, sgnl_fwelch_p3, sgnl_bwelch_n3, sgnl_bwelch_0, sgnl_bwelch_p3
    del sgnl_binent_0, sgnl_binent_f, sgnl_binent_b
    
    del trgt
    
    del sr_p1
    del sr_p1b5, sr_p1b4, sr_p1b3, sr_p1b2, sr_p1b1, sr_p1f1, sr_p1f2, sr_p1f3, sr_p1f4, sr_p1f5
    del sr_allwelch_n4, sr_allwelch_0, sr_allwelch_p4, sr_fwelch_n3, sr_fwelch_0, sr_fwelch_p3, sr_bwelch_n3, sr_bwelch_0, sr_bwelch_p3
    del sr_binent_0, sr_binent_f, sr_binent_b
    
    del sr_p2
    del sr_p2b5, sr_p2b4, sr_p2b3, sr_p2b2, sr_p2b1, sr_p2f1, sr_p2f2, sr_p2f3, sr_p2f4, sr_p2f5
    del sr, tr

100%|██████████| 10/10 [3:14:26<00:00, 1166.66s/it] 


In [7]:
for i in tqdm(range(0, 20)):
    sgnl_ndcs = batch_id_tst[i]
    
    # ------------------------------------------------
    sgnl_p1 = pdf_tst['signal'].iloc[sgnl_ndcs]
    p1_fill = sgnl_p1.mean()
    
    sgnl_p1b5 = sgnl_p1.shift(-5, fill_value=p1_fill)
    sgnl_p1b4 = sgnl_p1.shift(-4, fill_value=p1_fill)
    sgnl_p1b3 = sgnl_p1.shift(-3, fill_value=p1_fill)
    sgnl_p1b2 = sgnl_p1.shift(-2, fill_value=p1_fill)
    sgnl_p1b1 = sgnl_p1.shift(-1, fill_value=p1_fill)
    sgnl_p1f1 = sgnl_p1.shift(1, fill_value=p1_fill)
    sgnl_p1f2 = sgnl_p1.shift(2, fill_value=p1_fill)
    sgnl_p1f3 = sgnl_p1.shift(3, fill_value=p1_fill)
    sgnl_p1f4 = sgnl_p1.shift(4, fill_value=p1_fill)
    sgnl_p1f5 = sgnl_p1.shift(5, fill_value=p1_fill)
    
    sgnl_stck0 = np.stack([sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1, sgnl_p1, sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5], -1)
    sgnl_stckf = np.stack([sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5], -1)
    sgnl_stckb = np.stack([sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1], -1)
    
    sgnl_allwelch_n4 = pd.Series(np.array([wrapped_spkt_welch_density(sub, -4) for sub in sgnl_stck0]))
    sgnl_allwelch_0 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 0) for sub in sgnl_stck0]))
    sgnl_allwelch_p4 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 4) for sub in sgnl_stck0]))
    sgnl_fwelch_n3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, -3) for sub in sgnl_stckf]))
    sgnl_fwelch_0 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 0) for sub in sgnl_stckf]))
    sgnl_fwelch_p3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 3) for sub in sgnl_stckf]))
    sgnl_bwelch_n3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, -3) for sub in sgnl_stckb]))
    sgnl_bwelch_0 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 0) for sub in sgnl_stckb]))
    sgnl_bwelch_p3 = pd.Series(np.array([wrapped_spkt_welch_density(sub, 3) for sub in sgnl_stckb]))
    
    sgnl_binent_0 = pd.Series(np.array([feature_calculators.binned_entropy(sub, max_bins=10) for sub in sgnl_stck0]))
    sgnl_binent_f = pd.Series(np.array([feature_calculators.binned_entropy(sub, max_bins=10) for sub in sgnl_stckf]))
    sgnl_binent_b = pd.Series(np.array([feature_calculators.binned_entropy(sub, max_bins=10) for sub in sgnl_stckb]))

    sr_p1b5 = ts_utils.serie_stacker(sgnl_p1b5, step_size=step, block_size=wndw)
    sr_p1b4 = ts_utils.serie_stacker(sgnl_p1b4, step_size=step, block_size=wndw)
    sr_p1b3 = ts_utils.serie_stacker(sgnl_p1b3, step_size=step, block_size=wndw)
    sr_p1b2 = ts_utils.serie_stacker(sgnl_p1b2, step_size=step, block_size=wndw)
    sr_p1b1 = ts_utils.serie_stacker(sgnl_p1b1, step_size=step, block_size=wndw)
    sr_p1  = ts_utils.serie_stacker(sgnl_p1, step_size=step, block_size=wndw)
    sr_p1f1 = ts_utils.serie_stacker(sgnl_p1f1, step_size=step, block_size=wndw)
    sr_p1f2 = ts_utils.serie_stacker(sgnl_p1f2, step_size=step, block_size=wndw)
    sr_p1f3 = ts_utils.serie_stacker(sgnl_p1f3, step_size=step, block_size=wndw)
    sr_p1f4 = ts_utils.serie_stacker(sgnl_p1f4, step_size=step, block_size=wndw)
    sr_p1f5 = ts_utils.serie_stacker(sgnl_p1f5, step_size=step, block_size=wndw)
    
    sr_allwelch_n4 = ts_utils.serie_stacker(sgnl_allwelch_n4, step_size=step, block_size=wndw)
    sr_allwelch_0 = ts_utils.serie_stacker(sgnl_allwelch_0, step_size=step, block_size=wndw)
    sr_allwelch_p4 = ts_utils.serie_stacker(sgnl_allwelch_p4, step_size=step, block_size=wndw)
    sr_fwelch_n3 = ts_utils.serie_stacker(sgnl_fwelch_n3, step_size=step, block_size=wndw)
    sr_fwelch_0 = ts_utils.serie_stacker(sgnl_fwelch_0, step_size=step, block_size=wndw)
    sr_fwelch_p3 = ts_utils.serie_stacker(sgnl_fwelch_p3, step_size=step, block_size=wndw)
    sr_bwelch_n3 = ts_utils.serie_stacker(sgnl_bwelch_n3, step_size=step, block_size=wndw)
    sr_bwelch_0 = ts_utils.serie_stacker(sgnl_bwelch_0, step_size=step, block_size=wndw)
    sr_bwelch_p3 = ts_utils.serie_stacker(sgnl_bwelch_p3, step_size=step, block_size=wndw)
    
    sr_binent_0 = ts_utils.serie_stacker(sgnl_binent_0, step_size=step, block_size=wndw)
    sr_binent_f = ts_utils.serie_stacker(sgnl_binent_f, step_size=step, block_size=wndw)
    sr_binent_b = ts_utils.serie_stacker(sgnl_binent_b, step_size=step, block_size=wndw)
    
    # ------------------------------------------------
    sgnl_p2 = sgnl_p1**2
    p2_fill = sgnl_p2.mean()
    
    sgnl_p2b5 = sgnl_p2.shift(-5, fill_value=p2_fill)
    sgnl_p2b4 = sgnl_p2.shift(-4, fill_value=p2_fill)
    sgnl_p2b3 = sgnl_p2.shift(-3, fill_value=p2_fill)
    sgnl_p2b2 = sgnl_p2.shift(-2, fill_value=p2_fill)
    sgnl_p2b1 = sgnl_p2.shift(-1, fill_value=p2_fill)
    sgnl_p2f1 = sgnl_p2.shift(1, fill_value=p2_fill)
    sgnl_p2f2 = sgnl_p2.shift(2, fill_value=p2_fill)
    sgnl_p2f3 = sgnl_p2.shift(3, fill_value=p2_fill)
    sgnl_p2f4 = sgnl_p2.shift(4, fill_value=p2_fill)
    sgnl_p2f5 = sgnl_p2.shift(5, fill_value=p2_fill)
    
    sr_p2b5 = ts_utils.serie_stacker(sgnl_p2b5, step_size=step, block_size=wndw)
    sr_p2b4 = ts_utils.serie_stacker(sgnl_p2b4, step_size=step, block_size=wndw)
    sr_p2b3 = ts_utils.serie_stacker(sgnl_p2b3, step_size=step, block_size=wndw)
    sr_p2b2 = ts_utils.serie_stacker(sgnl_p2b2, step_size=step, block_size=wndw)
    sr_p2b1 = ts_utils.serie_stacker(sgnl_p2b1, step_size=step, block_size=wndw)
    sr_p2  = ts_utils.serie_stacker(sgnl_p2, step_size=step, block_size=wndw)
    sr_p2f1 = ts_utils.serie_stacker(sgnl_p2f1, step_size=step, block_size=wndw)
    sr_p2f2 = ts_utils.serie_stacker(sgnl_p2f2, step_size=step, block_size=wndw)
    sr_p2f3 = ts_utils.serie_stacker(sgnl_p2f3, step_size=step, block_size=wndw)
    sr_p2f4 = ts_utils.serie_stacker(sgnl_p2f4, step_size=step, block_size=wndw)
    sr_p2f5 = ts_utils.serie_stacker(sgnl_p2f5, step_size=step, block_size=wndw)
    
    sr = np.stack(
        [
            sr_p1, 
            sr_p1b5, sr_p1b4, sr_p1b3, sr_p1b2, sr_p1b1, sr_p1f1, sr_p1f2, sr_p1f3, sr_p1f4, sr_p1f5,
            sr_allwelch_n4, sr_allwelch_0, sr_allwelch_p4, sr_fwelch_n3, sr_fwelch_0, sr_fwelch_p3, sr_bwelch_n3, sr_bwelch_0, sr_bwelch_p3,
            sr_binent_0, sr_binent_f, sr_binent_b,
            sr_p2, 
            sr_p2b5, sr_p2b4, sr_p2b3, sr_p2b2, sr_p2b1, sr_p2f1, sr_p2f2, sr_p2f3, sr_p2f4, sr_p2f5,
        ],
        axis=2
    )
    
    bp.pack_ndarray_to_file(sr, '../input/tst_srs_dat_g{:d}_s{:d}_w{:d}_feat_basicwithnew.bp'.format(i, step, wndw))
    
    del sgnl_ndcs
    
    del sgnl_p1, sgnl_p2
    del sgnl_p1b5, sgnl_p1b4, sgnl_p1b3, sgnl_p1b2, sgnl_p1b1, sgnl_p1f1, sgnl_p1f2, sgnl_p1f3, sgnl_p1f4, sgnl_p1f5
    del sgnl_p2b5, sgnl_p2b4, sgnl_p2b3, sgnl_p2b2, sgnl_p2b1, sgnl_p2f1, sgnl_p2f2, sgnl_p2f3, sgnl_p2f4, sgnl_p2f5
    
    del sgnl_allwelch_n4, sgnl_allwelch_0, sgnl_allwelch_p4, sgnl_fwelch_n3, sgnl_fwelch_0, sgnl_fwelch_p3, sgnl_bwelch_n3, sgnl_bwelch_0, sgnl_bwelch_p3
    del sgnl_binent_0, sgnl_binent_f, sgnl_binent_b
    
    del sr_p1
    del sr_p1b5, sr_p1b4, sr_p1b3, sr_p1b2, sr_p1b1, sr_p1f1, sr_p1f2, sr_p1f3, sr_p1f4, sr_p1f5
    del sr_allwelch_n4, sr_allwelch_0, sr_allwelch_p4, sr_fwelch_n3, sr_fwelch_0, sr_fwelch_p3, sr_bwelch_n3, sr_bwelch_0, sr_bwelch_p3
    del sr_binent_0, sr_binent_f, sr_binent_b
    
    del sr_p2
    del sr_p2b5, sr_p2b4, sr_p2b3, sr_p2b2, sr_p2b1, sr_p2f1, sr_p2f2, sr_p2f3, sr_p2f4, sr_p2f5
    del sr


100%|██████████| 20/20 [50:26<00:00, 151.33s/it]
